## **April 2022 Tabular Playground Competition EDA**

* [Setup / Preliminaries](#setup)
* [Visual Exploration](#visualisations)
* [Dimensionality Reduction](#dimred)
    - [Original Data](#dimred1)
    - [Pivoted Data](#dimred2)
* [Generalised Additive Model](#gam)

<a id="setup"></a>
### **Setup / Preliminaries**

README: I'm commenting my code so you can hopefully follow my general thought process, but I haven't been motivated enough to include a full markdown commentary or polish the notebook.

In [3]:
# masking the warning messages
options(warn = -1)

In [4]:
# import any necessary libraries
# as per the (deleted) intro cell, the R environment is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
library(repr)
library(tidyverse)
library(corrplot)
library(GGally)
library(tidymodels)
library(FactoMineR)
library(factoextra)
library(mgcv)

In [5]:
# read in the data
train_data <- read_csv("../input/tabular-playground-series-apr-2022/train.csv", show_col_types = FALSE)
train_labels <- read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv", show_col_types = FALSE)
test_data <- read_csv("../input/tabular-playground-series-apr-2022/test.csv", show_col_types = FALSE)
sample_submission <- read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv", show_col_types = FALSE)

Recall the files and field description from the competition's *Data* tab:
- train.csv - the training set, comprising ~26,000 60-second recordings of thirteen biological sensors for almost one thousand experimental participants
    - sequence - a unique id for each sequence
    - subject - a unique id for the subject in the experiment
    - step - time step of the recording, in one second intervals
    - sensor_00 - sensor_12 - the value for each of the thirteen sensors at that time step

- train_labels.csv - the class label for each sequence.
    - sequence - the unique id for each sequence.
    - state - the state associated to each sequence. This is the target which you are trying to predict.

In [6]:
# look at the last couple of rows for each file
print("Training data")
tail(train_data)
print("Training labels")
tail(train_labels)
print("Testing data")
tail(test_data)

In [7]:
# check out the dimensions
dim(train_data)
dim(train_labels)
dim(test_data)

In [8]:
# and major summary statistics for the training data
summary(train_data)
summary(train_labels)

In [9]:
# confirm that there are no missing values
sum(is.na(train_data))

In [10]:
# check if there is an unbalanced target in the training set
table(train_labels$state)

options(repr.plot.width = 8, repr.plot.height = 7)

train_labels %>%
    ggplot(aes(x = state)) +
    geom_bar(fill = "chartreuse4") +
    scale_x_discrete(limits = c(0, 1), breaks = c(0, 1)) +
    labs(x = "State",
         y = "Count") +
    theme_classic()

<a id="visualisations"></a>
### **Visual Exploration**

In [11]:
# check out the distribution of sequences per subject
options(repr.plot.width = 14, repr.plot.height = 7)

train_data %>%
    group_by(subject) %>%
    distinct(sequence) %>%
    count() %>%
    ggplot(aes(x = n)) +
    geom_histogram(binwidth = 1, fill = "chartreuse4", col = "steelblue") +
    labs(x = "Number of sequences",
         y = "Number of subjects") +
    theme_classic() +
    theme(text = element_text(size = 15))

# ... and per subject per state
train_data_incl_target <- left_join(train_data, train_labels, by = "sequence")

train_data_incl_target %>%
    group_by(subject, state) %>%
    distinct(sequence) %>%
    count() %>%
    ggplot(aes(x = n, fill = factor(state))) +
    geom_histogram(binwidth = 1, col = "steelblue") +
    labs(x = "Number of sequences",
         y = "Number of subjects") +
    scale_fill_brewer(palette = "Greens", name = "State") +
    theme_classic() +
    theme(text = element_text(size = 15))

Now that's interesting. The single plot of the number of sequences for all subjects was approximately normally distributed, albeit with a long right tail. However, when grouped by state, we see that this is reflected only in the distibution of those recording state 0. The distribution of state 1 is not at all normal.

In [ ]:
# examine the distributions of each of the sensors, including broken up by State
long_train_data <- train_data_incl_target %>%
    pivot_longer(cols = starts_with("sensor"),
                names_to = "sensor",
                values_to = "reading")

options(repr.plot.width = 25, repr.plot.height = 15)

## original data with extreme values not shown
long_train_data %>%
    ggplot(aes(x = reading)) +
    geom_histogram(aes(y = ..density..), bins = 50, fill = "chartreuse4", col = "black") +
    facet_wrap(vars(sensor), ncol = 3, scales = "free_y") +
    xlim(-5, 5) +
    labs(x = "Number of sequences",
         y = "Number of subjects") +
    theme_classic()

options(repr.plot.width = 25, repr.plot.height = 15)

## and grouped by State
long_train_data %>%
    ggplot(aes(x = reading, fill = factor(state))) +
    geom_histogram(aes(y = ..density..), bins = 50, col = "steelblue") +
    facet_wrap(vars(sensor), ncol = 3, scales = "free_y") +
    xlim(-5, 5) +
    labs(x = "Number of sequences",
         y = "Number of subjects") +
    scale_fill_brewer(palette = "Greens", name = "State") +
    theme_classic()

In [ ]:
# check for any patterns in the aggregate statistics of sensor readings for each step
options(repr.plot.width = 15, repr.plot.height = 7)

long_train_data %>%
group_by(step, state) %>%
summarise(mean_reading = mean(reading),
          median_reading = median(reading),
          q25_reading = quantile(reading, probs = 0.25),
          q75_reading = quantile(reading, probs = 0.75)) %>%
    ggplot(aes(x = step, y = mean_reading, col = factor(state))) +
    geom_line() +
    geom_hline(yintercept = 0, linetype = 'dashed') +
    facet_wrap(vars(state)) +
    labs(x = "Step",
         y = "Average sensor reading") +
    scale_x_continuous(breaks = seq(0, 60, 5)) +
    scale_colour_brewer(palette = "Set1", name = "State") +
    theme_classic() +
    theme(legend.position = "none")

Again, it's quite interesting that the average value of the sensors is lower for State 0 even though both bounce around quite a bit during the 60 second period.

In [ ]:
# and examine the readings for each state faceted by sensor
options(repr.plot.width = 25, repr.plot.height = 15)

long_train_data %>%
group_by(step, state, sensor) %>%
summarise(mean_reading = mean(reading),
          median_reading = median(reading),
          q25_reading = quantile(reading, probs = 0.25),
          q75_reading = quantile(reading, probs = 0.75)) %>%
    ggplot(aes(x = step, y = mean_reading, col = factor(state))) +
    geom_line() +
    geom_hline(yintercept = 0, linetype = 'dashed') +
    facet_wrap(vars(sensor), scales = "free_y") +
    labs(x = "Step",
         y = "Average sensor reading") +
    scale_x_continuous(breaks = seq(0, 60, 10)) +
    scale_colour_brewer(palette = "Set1", name = "State") +
    theme_classic()

The biggest insight here is probably that the difference in average values by state is possibly driven by sensor 2 and to a lessor extent, sensor 4. Also, sensors 4, 10 and 12 tend to bounce around a little less than the others (excluding sensor 2 which appears to behave quite differently than the others).

In [ ]:
# visualise the linear correlations
# using spearman instead of the default pearson as it's unlikely that the data come from a bivariate normal distribution
corr_data <- train_data_incl_target %>% 
    dplyr::select(-c(sequence, subject))

options(repr.plot.width = 10, repr.plot.height = 10)

corr_mat <- round(cor(corr_data, method = "spearman"), 4)
corrplot::corrplot(corr_mat, method = "ellipse", type = "upper")

In [ ]:
# take a closer look at the relationships between features with the strongest correlations
options(repr.plot.width = 25, repr.plot.height = 15)

corr_data <- train_data_incl_target %>%
    dplyr::select(c(sensor_00, sensor_01, sensor_03, sensor_06, sensor_07, sensor_09, sensor_11, state))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 7)

corr_data %>%
    ggplot(aes(x = sensor_00, y = sensor_06, col = factor(state))) +
    geom_jitter() +
    scale_colour_brewer(palette = "Set1", name = "State") +
    theme_classic()

corr_data %>%
    ggplot(aes(x = sensor_01, y = sensor_06, col = factor(state))) +
    geom_jitter() +
    scale_colour_brewer(palette = "Set1", name = "State") +
    theme_classic()

corr_data %>%
    ggplot(aes(x = sensor_09, y = sensor_06, col = factor(state))) +
    geom_jitter() +
    scale_colour_brewer(palette = "Set1", name = "State") +
    theme_classic()

In [ ]:
corr_data %>%
    ggplot(aes(x = sensor_07, y = sensor_03, col = factor(state))) +
    geom_jitter() +
    scale_colour_brewer(palette = "Set1", name = "State") +
    theme_classic()

corr_data %>%
    ggplot(aes(x = sensor_11, y = sensor_03, col = factor(state))) +
    geom_jitter() +
    scale_colour_brewer(palette = "Set1", name = "State") +
    theme_classic()

This doesn't seem to tell us much, except that state 1 values tend to be clustered more tightly together and state 0 is more likely to contain extreme values.

In [ ]:
# as there were multiple rows for each sequence-subject combination, let's pivot the data to see what happens to the correlations
# this means that there will only be a single row for each sequence-subject combination
wide_train_data_incl_target <- train_data_incl_target %>%
    pivot_longer(!c(sequence, subject, step, state), names_to = "name", values_to = "value") %>%
    pivot_wider(names_from = c(step, name), values_from = value) %>%
    arrange(sequence, subject)

head(wide_train_data_incl_target)
# note that the column names are of the format [step]_[sensor]

In [ ]:
new_corr_data <- wide_train_data_incl_target
corr_mat <- round(cor(new_corr_data, method = "spearman"), 4)

In [ ]:
# turn the matrix into a dataframe and filter out any correlations less then 0.5
corr_mat_df <- as.data.frame(corr_mat) %>%
    mutate(row_name = rownames(corr_mat)) %>%
    pivot_longer(!row_name) %>%
    rename(
        col_name = name,
        correlation = value
    ) %>%
    filter(correlation >= 0.5) %>%
    arrange(desc(correlation))

# also remove any correlations where the feature is compared against itself
corr_mat_df <- corr_mat_df[corr_mat_df$row_name != corr_mat_df$col_name, ]
head(corr_mat_df)
dim(corr_mat_df)

The number of moderate to highly-correlated features in this pivoted dataframe suggests that dimensionality reduction could be useful. 

<a id="dimred"></a>
### **Dimensionality Reduction**

<a id="dimred1"></a>
#### **Original Data**

In [ ]:
# split the data, ensuring there are approximately equal numbers of each step in each (the dataset is big enough that it's probably reasonable to assume we'll get
# roughly equal proportions of each state)
set.seed(22)
data_split <- initial_split(train_data_incl_target[ , 3:17], prop = 0.75, strata = step)
train_data <- training(data_split)
val_data <- testing(data_split)

# scale the input features
train_data_sc <- as.data.frame(scale(train_data[-15]))
#train_data_sc$state <- train_data$state

In [ ]:
pca_mod <- PCA(train_data[, 1:14], scale.unit = TRUE, ncp = 10, graph = FALSE)

eig_val <- get_eigenvalue(pca_mod)
(eig_val)
fviz_eig(pca_mod, addlabels = TRUE, ylim = c(0, 20))

The first five PCs explain only 53% of the variance, but if we include all the PCs that have an eigenvalue of just below 1 (i.e. nine), we explain 74% of the variance. This is acceptable.

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 7)
fviz_pca_var(pca_mod, col.var = "cos2", repel = TRUE)
fviz_pca_var(pca_mod, col.var = "contrib", repel = TRUE)

In [ ]:
fviz_pca_ind(pca_mod, label = "none", habillage = train_data$state, select.ind = list(cos2 = 100))

These plots basically indicate the importance of features to the first two PCs. Basically, only 7 out of the 13 features are important.

<a id="gam"></a>
### **Generalised Additive Model**

In [ ]:
# http://www.sthda.com/english/wiki/wiki.php?id_contents=7851
# https://m-clark.github.io/generalized-additive-models/application.html